In [1]:
# 1. Import Necessary Libraries
import pandas as pd
from catboost import CatBoostRegressor, Pool
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from sklearn.metrics import mean_absolute_error

# 2. Load and Process Data (assuming the datasets are in the same directory)
df_true = pd.read_csv("./data/cleaned_actual.csv")
df_pred = pd.read_csv("./data/cleaned_forecasts.csv")



In [2]:
from pathlib import Path  
df_true= pd.read_csv("./data/cleaned_actual.csv")
df_pred= pd.read_csv("./data/cleaned_forecasts.csv")
df_pred
df_true = df_true.rename(columns={"Time": 'time', 
                                 "Load (kW)": "load_kw_true", 
                                 "Pressure_kpa": "pres_kpa_true",
                                 'Cloud Cover (%)': 'cld_pct_true',
                                 'Humidity (%)': 'hmd_pct_true',
                                 'Temperature (C)': 'temp_c_true',
                                 'Wind Direction (deg)': 'wd_deg_true',
                                 'Wind Speed (kmh)':'ws_kmh_true'})
df_pred = df_pred.rename(columns={"Time": 'time', 
                                 "Pressure (kpa)": "pres_kpa_pred",
                                 'Cloud Cover (%)': 'cld_pct_pred',
                                 'Temperature (C)': 'temp_c_pred',
                                 'Wind Direction (deg)': 'wd_deg_pred',
                                 'Wind Speed (kmh)':'ws_kmh_pred'})
result = pd.merge(df_true, df_pred, on="time")
result['time']= pd.to_datetime(result['time'])

df = result 
df = df.set_index('time')
df_true_lag = pd.DataFrame([])
df_true_lag['time'] = pd.to_datetime(df_true['time'])


for i in range(24, 169):
    df_true_lag['load_kw_lag' + str(i)] = df_true['load_kw'].shift(i)

df_true_lag['load_kw'] = df_true['load_kw']

# Adding forecasted temperature for the next hour
#df_true_lag['temp_c_pred_next_hour'] = df_pred['temperature_c'].shift(-24)

#df_true_lag['time'] = pd.to_datetime(df_true_lag['time'])

## Add forecasted temperatures for the next 24 to 48 hours as features
#for hour in range(24, 49):
#    shifted_temp = df_pred.set_index('time')['temperature_c'].shift(-hour).reset_index()
#    shifted_temp['time'] = pd.to_datetime(shifted_temp['time']) + pd.Timedelta(hours=hour)
#    df_true_lag = pd.merge(df_true_lag, shifted_temp, how='left', on='time', suffixes=('', f'_forecast_plus_{hour}h'))
    
df_true_lag

C:\Users\YUANJI JIN\AppData\Local\Temp\ipykernel_35604\3093580338.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_true_lag['load_kw_lag' + str(i)] = df_true['load_kw'].shift(i)
C:\Users\YUANJI JIN\AppData\Local\Temp\ipykernel_35604\3093580338.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_true_lag['load_kw_lag' + str(i)] = df_true['load_kw'].shift(i)
C:\Users\YUANJI JIN\AppData\Local\Temp\ipykernel_35604\3093580338.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling 

,time,load_kw_lag24,load_kw_lag25,load_kw_lag26,load_kw_lag27,load_kw_lag28,load_kw_lag29,load_kw_lag30,load_kw_lag31,load_kw_lag32,...,load_kw_lag160,load_kw_lag161,load_kw_lag162,load_kw_lag163,load_kw_lag164,load_kw_lag165,load_kw_lag166,load_kw_lag167,load_kw_lag168,load_kw
0,2017-03-18 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.031472e+06
1,2017-03-18 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.007206e+06
2,2017-03-18 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.861084e+05
3,2017-03-18 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.707610e+05
4,2017-03-18 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.622584e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33963,2021-01-31 03:00:00+00:00,9.766169e+05,988940.4321,1.005464e+06,1.022160e+06,1.035870e+06,1.063779e+06,1095986.932,1129619.932,1165659.932,...,1053517.932,1041089.432,1019824.932,9.929469e+05,9.731484e+05,9.682809e+05,9.676599e+05,965731.4321,968907.4321,9.774149e+05
33964,2021-01-31 04:00:00+00:00,9.766994e+05,976616.9321,9.889404e+05,1.005464e+06,1.022160e+06,1.035870e+06,1063779.432,1095986.932,1129619.932,...,1053177.432,1053517.932,1041089.432,1.019825e+06,9.929469e+05,9.731484e+05,9.682809e+05,967659.9321,965731.4321,9.717784e+05
33965,2021-01-31 05:00:00+00:00,9.827294e+05,976699.4321,9.766169e+05,9.889404e+05,1.005464e+06,1.022160e+06,1035869.932,1063779.432,1095986.932,...,1047378.432,1053177.432,1053517.932,1.041089e+06,1.019825e+06,9.929469e+05,9.731484e+05,968280.9321,967659.9321,9.738789e+05
33966,2021-01-31 06:00:00+00:00,9.922074e+05,982729.4321,9.766994e+05,9.766169e+05,9.889404e+05,1.005464e+06,1022160.432,1035869.932,1063779.432,...,1045147.932,1047378.432,1053177.432,1.053518e+06,1.041089e+06,1.019825e+06,9.929469e+05,973148.4321,968280.9321,9.792199e+05


In [3]:
# 3. Define the Training Function
def train_best_catboost_model(train_data, val_data):
    # Select all features except the target and 'time' for training
    X_train = train_data.drop(['load_kw', 'time'], axis=1)
    y_train = train_data['load_kw']
    X_val = val_data.drop(['load_kw', 'time'], axis=1)
    y_val = val_data['load_kw']
    
    train_pool = Pool(data=X_train, label=y_train)
    val_pool = Pool(data=X_val, label=y_val)

    def catboost_objective(params):
        model = CatBoostRegressor(
            iterations=100, 
            depth=int(params['depth']),
            learning_rate=params['learning_rate'],
            subsample=params['subsample'],
            colsample_bylevel=params['colsample_bylevel'],
            loss_function='MAE',
            eval_metric='MAE',
            verbose=False
        )
        model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=10)
        
        val_preds = model.predict(val_pool)
        mae = mean_absolute_error(y_val, val_preds)
        return {'loss': mae, 'status': STATUS_OK}

    catboost_space = {
        'depth': hp.choice('depth', range(2, 11, 1)),
        'learning_rate': hp.uniform('learning_rate', 0.1, 0.3),
        'subsample': hp.uniform('subsample', 0.5, 1.0),
        'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1.0),
    }

    trials = Trials()
    best = fmin(fn=catboost_objective, space=catboost_space, algo=tpe.suggest, max_evals=100, trials=trials)
    
    best_params = space_eval(catboost_space, best)
    best_model = CatBoostRegressor(
        iterations=100,
        depth=best_params['depth'],
        learning_rate=best_params['learning_rate'],
        subsample=best_params['subsample'],
        colsample_bylevel=best_params['colsample_bylevel'],
        loss_function='MAE',
        eval_metric='MAE',
        verbose=False
    )
    best_model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=10)
    
    return best_model, best_params, trials.best_trial['result']['loss']
# Splitting data into train and validation sets
n = len(df_true_lag)
train_data = df_true_lag[:int(n*0.9)]
val_data = df_true_lag[int(n*0.9):]

# 4. Train the Model
best_model, best_params, best_loss = train_best_catboost_model(train_data, val_data)




100%|██████████| 100/100 [02:35<00:00,  1.55s/trial, best loss: 18691.4270657037] 


In [4]:
best_model

In [5]:
best_params

{'colsample_bylevel': 0.8304530576058512,
 'depth': 6,
 'learning_rate': 0.17400357181596038,
 'subsample': 0.9466693131507607}

In [6]:
best_loss

18691.4270657037